# Data processing course assignments
---

In [1]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import asc, lit
import re

In [2]:
#conf = SparkConf().setAppName('Assignment').setMaster('local')
#sc = SparkContext(conf=conf)

Introducing `SparkSession`, it is a combination of the previous versions `SparkConf`, `SparkContext`, `SQLContext`, and `HiveContext`.

In [3]:
spark = SparkSession \
    .builder \
    .appName("Python Spark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

## Ejercicio 0.

## Ejercicio 1.
#### Leer el archivo data/containers.csv y contar el número de líneas.
---

Obteniendo los datos del archivo csv.

In [4]:
ls

Assignments.ipynb*  data_processing_course-master/  spark-warehouse/
containers.csv*     PySpark.ipynb*                  textMining/


In [5]:
path = "/home/jovyan/work/data_processing_course-master/assignments/data/containers.csv"
csvdata = spark.read.format("csv").option("header", "false").load(path)

In [6]:
# Comprobando obtención de los datos.
csvdata.take(2)

[Row(_c0='ship_imo;ship_name;country;departure;container_id;container_type;container_group;net_weight;gross_weight;owner;declared;contact;customs_ok'),
 Row(_c0='AMC1861710;Jayden;BD;201602183;FCUK1755843;4960;28VH;44804866.62;2240243.33;Streich-Wilkinson;Music')]

Contando el número de líneas del archivo csv.

In [7]:
csvdata.count()

614

## Ejercicio 2.
#### Leer el archivo data/containers.csv y filtrar aquellos contenedores cuyo ship_imo es DEJ1128330 y el grupo del contenedor es 22P1. Guardar los resultados en un archivo de texto en resultados/resutado_2.
---

In [8]:
print(type(csvdata))
datardd = csvdata.rdd
print(type(datardd))

datasplit = datardd.map(lambda row: row[0].split(";"))

datasplit.take(2)

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.rdd.RDD'>


[['ship_imo',
  'ship_name',
  'country',
  'departure',
  'container_id',
  'container_type',
  'container_group',
  'net_weight',
  'gross_weight',
  'owner',
  'declared',
  'contact',
  'customs_ok'],
 ['AMC1861710',
  'Jayden',
  'BD',
  '201602183',
  'FCUK1755843',
  '4960',
  '28VH',
  '44804866.62',
  '2240243.33',
  'Streich-Wilkinson',
  'Music']]

In [9]:
# datafiltered = datasplit.filter(lambda row: row == '22P1')
datasplit.map(lambda row: row[0]).take(10)

['ship_imo',
 'AMC1861710',
 'POG1615575',
 'SQH1155999',
 'JCI1797526',
 'MBV1836745',
 'GYR1192020',
 'GLV1922612',
 'NLH1771681',
 'FUS1202266']

In [10]:
datasplit.filter(lambda row: row[6] == '22P1' and row[0] == 'DEJ1128330').count()

2

In [122]:
datasplit.filter(lambda row: row[6] == '22P1' and row[0] == 'DEJ1128330').take(2)

[['DEJ1128330',
  'Tiara',
  'GP',
  '2016021818',
  'GYFD1228113',
  '20PF',
  '22P1',
  '51503716.88',
  '5150371.69',
  'Armstrong-Goldner',
  'Automotive'],
 ['DEJ1128330',
  'Tiara',
  'GP',
  '2016021818',
  'MBPF1909627',
  '24H2',
  '22P1',
  '37266600.88',
  '1863330.04',
  'Lehner-Hamill',
  'Jewelery']]

Guardando los datos como archivo de texto

In [13]:
x1 = datasplit.filter(lambda row: row[6] == '22P1' and row[0] == 'DEJ1128330')
x1.saveAsTextFile('data_processing_course-master/assignments/resultados/resultado_2')

## Ejercicio 3.
#### Leer el archivo data/containers.csv y convertir a formato Parquet. Recuerda que puedes hacer uso de la funcion parse_container en helpers.py tal y como vimos en clase. Guarda los resultados en resultados/resultado_3.
---

Using `SparkSession` to create a Dataframe from a csv. I am using a Dataframe instead of a RDD, because are RDDs are schemaless, and Dataframes are close to Parquet files structure.

In [14]:
data3 = spark.read.csv(path, header=True, sep=';') #Usamos el mismo path creado previamente

In [15]:
data3.show(3)

+----------+-----------------+-------+----------+------------+--------------+---------------+-----------+------------+-----------------+--------------------+--------------------+----------+
|  ship_imo|        ship_name|country| departure|container_id|container_type|container_group| net_weight|gross_weight|            owner|            declared|             contact|customs_ok|
+----------+-----------------+-------+----------+------------+--------------+---------------+-----------+------------+-----------------+--------------------+--------------------+----------+
|AMC1861710|           Jayden|     BD| 201602183| FCUK1755843|          4960|           28VH|44804866.62|  2240243.33|Streich-Wilkinson|Music, Tools, Aut...|octavia@stammbedn...|      true|
|POG1615575|Lake Eribertoland|     CR|2016021611| PDXW1549639|          28VH|           8888|16681047.32|   500431.42|  Senger and Sons|   Movies & Jewelery|cindy.dubuque@rob...|      true|
|SQH1155999|           Aileen|     GN| 201602191| 

In [16]:
cd data_processing_course-master/assignments/resultados

/home/jovyan/work/data_processing_course-master/assignments/resultados


In [17]:
data3.write.parquet("data_processing_course-master/assignments/resultados/resultado_3")

In [18]:
data3.count()

613

## Ejercicio 4.
#### Lee el archivo de Parquet guardado en el ejercicio 3 y filtra los barcos que tienen al menos un contenedor donde la columna customs_ok es igual a false. Extrae una lista con los identificadores de barco, ship_imo, sin duplicados y ordenados alfabéticamente, en formato json.
---

In [19]:
data4 = spark.read.parquet("data_processing_course-master/assignments/resultados/resultado_3")

In [20]:
data4.count()

613

In [21]:
data4.columns

['ship_imo',
 'ship_name',
 'country',
 'departure',
 'container_id',
 'container_type',
 'container_group',
 'net_weight',
 'gross_weight',
 'owner',
 'declared',
 'contact',
 'customs_ok']

In [22]:
data4.createOrReplaceTempView("data")

In [23]:
spark.sql("SELECT DISTINCT ship_imo FROM data WHERE customs_ok = 'false'").show()

+----------+
|  ship_imo|
+----------+
|KSP1096387|
|GYR1192020|
|JET1053895|
|SQH1155999|
|NLH1771681|
|JCI1797526|
|GEU1548633|
|AEY1108363|
|IWE1254579|
|AMC1861710|
|POG1615575|
|MBV1836745|
|GLV1922612|
|YZX1455509|
|TCU1641123|
|JMP1637582|
|DEJ1128330|
|RYP1117603|
|FUS1202266|
|NCZ1777367|
+----------+



In [24]:
data4 = spark.sql("SELECT DISTINCT ship_imo FROM data WHERE customs_ok = 'false'")

In [25]:
spark.sql("SELECT DISTINCT ship_imo FROM data WHERE customs_ok = 'false'").count()

20

In [26]:
type(data4)

pyspark.sql.dataframe.DataFrame

In [27]:
data4.sort(asc("ship_imo")).collect()

[Row(ship_imo='AEY1108363'),
 Row(ship_imo='AMC1861710'),
 Row(ship_imo='DEJ1128330'),
 Row(ship_imo='FUS1202266'),
 Row(ship_imo='GEU1548633'),
 Row(ship_imo='GLV1922612'),
 Row(ship_imo='GYR1192020'),
 Row(ship_imo='IWE1254579'),
 Row(ship_imo='JCI1797526'),
 Row(ship_imo='JET1053895'),
 Row(ship_imo='JMP1637582'),
 Row(ship_imo='KSP1096387'),
 Row(ship_imo='MBV1836745'),
 Row(ship_imo='NCZ1777367'),
 Row(ship_imo='NLH1771681'),
 Row(ship_imo='POG1615575'),
 Row(ship_imo='RYP1117603'),
 Row(ship_imo='SQH1155999'),
 Row(ship_imo='TCU1641123'),
 Row(ship_imo='YZX1455509')]

In [28]:
data4.sort(asc("ship_imo")).write.json("data_processing_course-master/assignments/resultados/resultado_4")

## Ejercicio 5.
#### Crea una UDF para validar el código de identificación del contenedor container_id. Para simplificar la validación, daremos como válidos aquellos códigos compuestos de 3 letras para el propietario, 1 letra para la categoría, 6 números y 1 dígito de control. Devuelve un DataFrame con los campos: ship_imo, container_id, propietario, categoria, numero_serie y digito_control.
---

In [29]:
spark.sql("SELECT DISTINCT container_id FROM data ").show()

+------------+
|container_id|
+------------+
| DXTQ1407119|
| VXNB1938296|
| OVMU1118217|
| EAQO1539643|
| SGQH1799946|
| SWXT1708984|
| EDBR1562470|
| FCMB1487245|
| UBRI1681197|
| LFTG1322014|
| RAUX1713695|
| QKOJ1756061|
| JVFH1614514|
| KETX1362337|
| KUOG1927848|
| GSVC1467358|
| JXRI1226202|
| QZKL1853985|
| HDGM1122708|
| ZBJH1066313|
+------------+
only showing top 20 rows



Getting familiar with Python re package

In [30]:
test5 = spark.sql("SELECT DISTINCT container_id FROM data ").take(2)
type(test5)
type(test5[0])
print(str(test5))

[Row(container_id='DXTQ1407119'), Row(container_id='VXNB1938296')]


In [31]:
containeridre = re.search(r'\w\w\w\w\d\d\d\d\d\d\d', str(test5[0]))

In [32]:
if containeridre:
    print('ez pz')
else:
    pass

ez pz


Creating function to validate container ID

In [40]:
def validateID(containerID):
    containeridre = re.search(r'\w\w\w\w\d\d\d\d\d\d\d', str(containerID))
    #nonvalid = []
    if containeridre:
        return("Container ID valid")
    else:
        #nonvalid.append(str(containerID))
        return("----- Container ID not valid: " + str(containerID) + '-----')

In [78]:
data5 = spark.sql("SELECT DISTINCT container_id FROM data ")

In [42]:
data5col = data5.select("container_id")

In [43]:
data5col.rdd.take(2)

[Row(container_id='DXTQ1407119'), Row(container_id='VXNB1938296')]

In [44]:
validateID('DXTQ1407119')

'Container ID valid'

In [45]:
data5.rdd.map(lambda row: validateID(str(row))).collect()

['Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container ID valid',
 'Container

Contenedores que no tienen un código válido

In [80]:
data5.rdd.map(lambda row: validateID(str(row))).filter(lambda x: str(x) != 'Container ID valid').collect()

['----- Container ID not valid: Row(container_id=None)-----',
 "----- Container ID not valid: Row(container_id='JMYG190Z978')-----",
 "----- Container ID not valid: Row(container_id='GJFL14A2798')-----",
 "----- Container ID not valid: Row(container_id='CTVU1506A832')-----",
 "----- Container ID not valid: Row(container_id='DUKF166276')-----"]

In [129]:
testdata = spark.sql("SELECT DISTINCT container_id FROM data ")

AttributeError: 'NoneType' object has no attribute 'show'

## Ejercicio 6. 
#### Extrae una lista con peso total de cada barco, net_weight, sumando cada contenedor y agrupado por los campos ship_imo y container_group. Devuelve un DataFrame con la siguiente estructura: ship_imo, ship_name, container, total_net_weight.
---

In [81]:
data6 = data3

In [82]:
data6.take(2)

[Row(ship_imo='AMC1861710', ship_name='Jayden', country='BD', departure='201602183', container_id='FCUK1755843', container_type='4960', container_group='28VH', net_weight='44804866.62', gross_weight='2240243.33', owner='Streich-Wilkinson', declared='Music, Tools, Automotive & Health', contact='octavia@stammbednar.name', customs_ok='true'),
 Row(ship_imo='POG1615575', ship_name='Lake Eribertoland', country='CR', departure='2016021611', container_id='PDXW1549639', container_type='28VH', container_group='8888', net_weight='16681047.32', gross_weight='500431.42', owner='Senger and Sons', declared='Movies & Jewelery', contact='cindy.dubuque@roberts.org', customs_ok='true')]

In [83]:
data6new = data6.drop("country", "departure", "container_type", \
                      "container_group", "gross_weight", "owner", \
                      "declared", "customs_ok", "contact", "net_weight")

In [84]:
data6new.show()

+----------+------------------+------------+
|  ship_imo|         ship_name|container_id|
+----------+------------------+------------+
|AMC1861710|            Jayden| FCUK1755843|
|POG1615575| Lake Eribertoland| PDXW1549639|
|SQH1155999|            Aileen| PLKO1661930|
|JCI1797526|          Herminio| BXMT1827488|
|MBV1836745|Port Guiseppeburgh| JYIE1892741|
|GYR1192020|         Emardland| LARQ1499256|
|GLV1922612|           Eulalia| ARDX1463154|
|NLH1771681|       Port Noemie| JFPX1246669|
|FUS1202266|  East Mustafaland| ICAV1235470|
|GLV1922612|           Eulalia| KEVU1145768|
|IWE1254579|      North Creola| VDUQ1801278|
|JET1053895|             Jamil| CXZN1286843|
|KSP1096387|             Wiley| YAZN1142572|
|GYR1192020|         Emardland| HVFU1799048|
|GYR1192020|         Emardland| ROML1055099|
|JMP1637582|East Zechariahland| LONM1299749|
|TCU1641123|     New Margarete| XKAO1357085|
|MBV1836745|Port Guiseppeburgh| JYPA1889923|
|POG1615575| Lake Eribertoland| GKXC1181753|
|AEY110836

In [85]:
data6new = data6new.withColumn("total_net_weight", lit(0))

In [86]:
spark.sql("SELECT DISTINCT ship_name FROM data").show()

+------------------+
|         ship_name|
+------------------+
|         Emardland|
|             Tiara|
|East Zechariahland|
| Lake Eribertoland|
|             Jamil|
|           Eulalia|
|  East Mustafaland|
|   New Santinoberg|
|            Aileen|
|            Jayden|
|        Marksshire|
|      North Creola|
|       Port Noemie|
|     New Margarete|
|             Wiley|
|   East Savanahton|
|          Herminio|
|Port Guiseppeburgh|
|           Lomaton|
|             Juana|
+------------------+



Mismo filtrado de datos en dos formas distintas:

In [87]:
spark.sql("SELECT DISTINCT container_id FROM data WHERE ship_name='Emardland' ").count()

26

In [105]:
data6.filter(data6['ship_name']=='Emardland').count()

26

In [124]:
def test(x):
    return str(x) + 'asd'

AttributeError: 'NoneType' object has no attribute 'collect'

## Ejercicio 7.
#### Guarda los resultados del ejercicio anterior en formato Parquet.
---

## Ejercicio 8.
#### ¿En qué casos crees que es más eficiente utilizar formatos como Parquet? ¿Existe alguna desventaja frente a formatos de texto como CSV?
---

In [ ]:
Precio en AWS

## Ejercicio 9.
#### ¿Es posible procesar XML mediante Spark? ¿Existe alguna restricción por la cual no sea eficiente procesar un único archivo en multiples nodos? ¿Se te ocurre alguna posible solución para trocear archivos suficientemente grandes? ¿Existe la misma problemática con otros formatos de texto como JSON?
---

Sería posible mediante un librería externa (spark.read.xml que viene de databricks)

## Ejercicio 10.
#### Spark SQL tiene una función denominada avg que se utiliza para calcular el promedio de un conjunto de valores ¿Por qué los autores han creado esta función en lugar de usar el API estándar de Python o Scala?
---